<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/AutoRegressieveModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import re
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [4]:
all_data = []
resp = urlopen("https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
shakespear_100k = resp.read()
shakespear_100k = shakespear_100k.decode('utf-8').lower()
device = torch.device("cuda")


In [5]:
vocab2index = {}

for char in shakespear_100k:
  if char not in vocab2index:
    vocab2index[char] = len(vocab2index)

index2Vocab={}

for k, v in vocab2index.items():
  index2Vocab[v]=k


print("Vocab Size ", len(vocab2index))
print("Total text size ", len(shakespear_100k))



Vocab Size  39
Total text size  1115394


In [6]:
class AutoRegressiveDataset(Dataset):

  def __init__(self, large_string, max_chunk=500):
    self.doc = large_string
    self.max_chunk = max_chunk

  def __len__(self):
    return (len(self.doc)-1) // self.max_chunk

  def __getitem__(self, idx):
    start = idx * self.max_chunk
    sub_string = self.doc[start:start+self.max_chunk]
    x = [vocab2index(c) for c in sub_string]
    sub_string_y = self.doc[start+1:start+self.max_chunk+1]
    y = [vocab2index(c) for c in sub_string_y]
    

In [7]:
def step(x_in, h_prev):
  if len(x_in.shape) == 1:
    x_in = self.embd(x_in)
  
  if h_prevs is None:
    h_prevs = self.initHiddenStates(x_in.shape[0])


  # for GRUCell there is only one input that is send as the hidden state to the next time step process and as an output from this layer. 
  for l in range(len(self.layers)):  
    h_prev = h_prevs[l]
    h = self.norms[l](self.layers[l](x_in, h_prev))
    h_prevs[l]=h
    x_in = h

  return self.pred_class(x_in)


In [8]:
class AutoRegressiveModule(nn.Module):
  
  def __init__(self, num_embeddings, embd_size, hidden_size, layers=1):
    super(AutoRegressiveModule, self).__init__()
    self.hidden_size = hidden_size
    self.embd = nn.Embedding(num_embeddings, embd_size)
    self.layers = nn.ModuleList( [nn.GRUCell(embd_size, hidden_size)] + [nn.GRUCell(hidden_size, hidden_size) for i in range(layers-1)])
    self.norms = nn.ModuleList([nn.LayerNorm(hidden_size) for i in range(layers)])
    self.pred_class = nn.Sequential(
        nn.Linear(hidden_size, hidden_size), 
        nn.LeakyReLU(),
        nn.LayerNorm(hidden_size),
        nn.Linear(hidden_size, num_embeddings)   
    )

  def initHiddenStates(self, B):
    return [torch.zeros(B, self.hidden_size, device=device) for _ in range(len(self.layers))]

  def forward(self, input):
    B = input.size(0)
    T = input.size(1)
    x = self.embd(input)
    h_prevs = self.initHiddenStates(B)
    
    last_activations = []
    for t in range(T):
      x_in = x[:,t, :]
      last_activations.append(self.step(x_in, h_prevs))
    
    last_activations = torch.stack(last_activations, dim=1)
    return last_activations


